In [1]:
import pyarrow.parquet as pq

In [3]:
df = pq.read_table('../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/subset_cmg/test-00000-of-00002-983a1050cd3c058c.parquet').to_pandas()

In [4]:
df.tail()

,author,date,timezone,hash,message,mods,language,license,repo,original_message
102163,887020,11.02.2022 22:24:43,-3600,e960d02ab094518ddb41454e16bbd0ef601ecbb9,Fix context merging\n* Fix context merging\nWh...,"[{'change_type': 'MODIFY', 'old_path': 'CHANGE...",Ruby,MIT License,getsentry/sentry-ruby,Fix context merging #1722 (#1724)\n\n* Fix con...
102164,887022,17.03.2022 22:42:36,0,1e71de02e9e09bee034536ed9e44042be4ea3f22,Explicitly use https as github-sourced gems' p...,"[{'change_type': 'MODIFY', 'old_path': 'sentry...",Ruby,MIT License,getsentry/sentry-ruby,Explicitly use https as github-sourced gems' p...
102165,887022,18.03.2022 11:37:38,0,ff4878867d2d796f2d147986c7c3dec10be8800b,Ignore internal Sidekiq::JobRetry::Skip exception,"[{'change_type': 'MODIFY', 'old_path': 'sentry...",Ruby,MIT License,getsentry/sentry-ruby,Ignore internal Sidekiq::JobRetry::Skip except...
102166,886973,18.03.2022 15:15:43,-3600,723b1c9895105eefa2becaad630e442f6333a758,fix(perf): TransactionEvent should not set lev...,"[{'change_type': 'MODIFY', 'old_path': 'sentry...",Ruby,MIT License,getsentry/sentry-ruby,fix(perf): TransactionEvent should not set lev...
102167,886973,21.03.2022 12:05:03,-3600,9ed47c9f9f87dfdcd1a38fa05451d2ac3a1b8f3d,fix: Set last_event_id only for error events,"[{'change_type': 'MODIFY', 'old_path': 'CHANGE...",Ruby,MIT License,getsentry/sentry-ruby,fix: Set last_event_id only for error events (...


In [61]:
df['mods'][0][0]['change_type']

'MODIFY'

In [19]:
for index, row in df.iterrows():
    print(row['message'])
    break

Fixed loading terms from some GR pages that had a space after the class type


In [5]:
languages = ['Java', 'Python', 'C#', 'C++', 'JavaScript']
language_dict = {language: 0 for language in languages}

In [1]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm

# Create empty DataFrames for each language
# languages = ['Java', 'Python', 'C++', 'C#', 'JavaScript']
languages = ['Python', 'JavaScript']
dfs = {lang: [] for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=1000, desc=lang) for lang in languages}

folder_path = '../rag/datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/subset_cmg'  # 文件夹的路径

files = glob.glob(os.path.join(folder_path, 'test*'))

for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages[:]:
        # Filter rows where language column matches the current language
        lang_df = df[(df['language'] == lang) & (df['mods'].apply(len) == 1) & (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') & (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 3000))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            diff = row['mods'][0]
            old_path = 'a/' + diff['old_path']
            new_path = 'b/' + diff['new_path']
            diff_content = diff['diff']  # assume diff_content is an empty string
            item = {
                'msg': row['message'],
                'diff': f"diff --git {old_path} {new_path} {diff_content}",
                'lang': lang
            }
            dfs[lang].append(item)
            bars[lang].update(1)

            # Check if the language has reached 100,000 rows
            if len(dfs[lang]) >= 1000:
                print(f"Reached 1000 rows for {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break
    # Break out of the loop if all languages have reached 100,000 rows
    if not languages:
        break

Python:   0%|          | 0/1000 [00:00<?, ?it/s]

JavaScript:   0%|          | 0/1000 [00:00<?, ?it/s]

Reached 1000 rows for Python
Reached 1000 rows for JavaScript


In [4]:
import json
data = []
while any(value < 1000 for value in language_dict.values()):
    for index, row in df.iterrows():
        language = row['language']
        if language in languages:
            
            if language_dict[language] >= 1000 or len(row['mods']) != 1 or row['mods'][0]['change_type'] != 'MODIFY' or len(f"diff --git a/{row['mods'][0]['old_path']} b/{row['mods'][0]['new_path']} {row['mods'][0]['diff']}") > 3000:
                continue
            language_dict[language] += 1

            item = {
                'msg': row['message'],
                'diff': f"diff --git a/{row['mods'][0]['old_path']} b/{row['mods'][0]['new_path']} {row['mods'][0]['diff']}",
                'lang': language
            }
            data.append(item)

import random
random.seed(42)
random.shuffle(data)

with open('../data/chronicle_data_dev.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, indent=4)


NameError: name 'language_dict' is not defined

In [5]:
import json

# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]

import random
random.seed(42)
random.shuffle(all_items)

# Dump the data to a JSON file
with open('../data/chronicle/chronicle_test_data_dev.json', 'w') as f:
    json.dump(all_items, f, indent=4)

In [3]:
len(all_items)

2000

In [66]:
# 遍历每个item
for item in data:
    diff = item['diff']
    new_diff = f"diff --git {diff}"

    item['diff'] = new_diff

# 将更新后的数据写入新的JSON文件
with open('../data/chronicle_data.json', 'w', encoding='UTF-8') as file:
    json.dump(data, file, indent=4)

In [1]:
len(df)

NameError: name 'df' is not defined